In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 37.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747921 sha256=b0d0bcf56d9e487016904722a0580548d03c557774f483122dbc45e47dc21c6b
  Stored in directory: /home/codespace/.cache/pip/wheels/8f/cb/c0/cc57eb1bf0f9dc87cdaf2b0dbac49e58a210ff68d21d6fc709
Successfully built pyspark

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,explode,sum

In [3]:
spark = SparkSession.builder.appName("CustomerOrderPipeline").getOrCreate()

25/05/14 08:44:30 WARN Utils: Your hostname, codespaces-f8e69a resolves to a loopback address: 127.0.0.1; using 10.0.1.40 instead (on interface eth0)
25/05/14 08:44:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/14 08:44:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.option("multiline","true").json("customer_orders.json")
df = df.withColumn("order",explode("orders"))
flat_df = df.select(
    col("customer_id"),
    col("name"),
    col("email"),
    col("order.order_id").alias("order_id"),
    col("order.amount").alias("amount"),
    col("order.date").alias("order_date")
)
flat_df.show()

+-----------+-----+-----------------+--------+------+----------+
|customer_id| name|            email|order_id|amount|order_date|
+-----------+-----+-----------------+--------+------+----------+
|        101|Rahul|rahul@example.com|    5001|  2500|2024-05-01|
|        101|Rahul|rahul@example.com|    5002|  1800|2024-05-10|
|        102|Meena|meena@example.com|    5003|  3200|2024-05-05|
+-----------+-----+-----------------+--------+------+----------+



In [5]:
customers = flat_df.select("customer_id","name","email").distinct()
orders = flat_df.select("order_id","customer_id","amount","order_date")

orders.write.partitionBy("customer_id").mode("overwrite").csv("orders_partitioned")

In [6]:
total_spent = orders.groupBy("customer_id").agg(sum("amount").alias("total_spent"))
total_spent.show()

+-----------+-----------+
|customer_id|total_spent|
+-----------+-----------+
|        101|       4300|
|        102|       3200|
+-----------+-----------+



25/05/14 08:44:47 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
